In [1]:
import os
import pandas as pd
import gspread
import googlemaps
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
load_dotenv()

SERVICE_ACCOUNT_JSON = os.getenv('SERVICE_ACCOUNT_JSON')
SHEET_KEY = os.getenv('SHEET_KEY')
MAPS_API = os.getenv('MAPS_API')
CONN_STR = os.getenv('CONN_STR')

In [3]:
# Access Google Sheets
gc = gspread.service_account(filename=SERVICE_ACCOUNT_JSON)

sh = gc.open_by_key(SHEET_KEY)

wksh = sh.sheet1

In [4]:
# Convert Sheets to Pandas Dataframe
df = pd.DataFrame(wksh.get_all_records())
# Remove excess information
# TODO : implement a way to automatically determine what the last entry is.
tot_entries = 9
df = df.iloc[:tot_entries]

In [5]:
df

,Company Name,Address,Phone Number,Website,Instagram,Food Menu,Table Seating,Wifi,Outlets,Parking
0,Neat Coffee,"1922 Pomona Ave, Costa Mesa, CA 92627",(949) 873-6328,https://www.neat.coffee/,https://www.instagram.com/neat.coffee/,"Hot food, Baked goods",Yes,Yes,Yes,Yes
1,Moongoat Coffee,"1985 Placentia Ave, Costa Mesa, CA 92627",(949) 612-2875,moongoat.online,https://www.instagram.com/moongoatcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Limited
2,Nüdae Coffee,"1125 Victoria St # U, Costa Mesa, CA 92627",(408) 667-9051,https://www.nudaecoffee.com/,https://www.instagram.com/nudaecoffee/,"Hot food, Baked goods",Limited,Yes,Yes,Yes
3,Lion & Lamb Coffee Roasters,"3186 Pullman St, Costa Mesa, CA 92626",,https://lionandlambcoffee.com/,www.instagram.com/lionandlambcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Yes
4,Work in Progress,"2944 Randolph Av # 1, Costa Mesa, CA 92626",,https://www.wipgoods.com/,https://www.instagram.com/wipgoods/,"Hot food, Baked goods",Limited,Yes,Very limited,Limited
5,In-sīt Coffee,"891 Baker St B19, Costa Mesa, CA 92626",(714) 852-3040,https://www.insitcoffee.com/,https://www.instagram.com/insit_coffee,Baked goods only,Very limited,Yes,Very limited,Limited
6,Buenas Coffee,"150 Paularino Ave Building B, Costa Mesa, CA 9...",,https://www.buenas.coffee/,https://www.instagram.com/buenas.coffee,Baked goods only,Limited,Yes,Yes,Yes
7,Portola Coffee Roasters,"3313 Hyland Ave STE B2, Costa Mesa, CA 92626",(714) 656-7996,https://www.portolacoffee.com/,https://www.instagram.com/portolacoffee,Baked goods only,Yes,Yes,Yes,Yes
8,Philz Coffee,"3420 Bristol St, Costa Mesa, CA 92626",(714) 486-2731,https://philzcoffee.com/,https://www.instagram.com/philzcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Yes


In [6]:
# Access Google Maps
gmaps = googlemaps.Client(key=MAPS_API)

In [7]:
# Function for getting hours given an address
# TODO : implement getting photos, phone number, website as well.
def get_hours(address):
    geocode = gmaps.geocode(address)
    
    places_result = gmaps.places(query='coffee shop', location=(geocode[0]['geometry']['location']['lat'], geocode[0]['geometry']['location']['lng']))
    
    place_id = places_result['results'][0]['place_id']
    place_details = gmaps.place(place_id=place_id, fields=['name', 'formatted_address', 'opening_hours'])
    
    if 'opening_hours' in place_details['result']:
        return(place_details['result']['opening_hours']['weekday_text'])
    else:
        return('Opening hours not available')

In [8]:
# Using Address, create new column in dataframe from Hours
df['Hours'] = df['Address'].apply(get_hours)

In [9]:
df

,Company Name,Address,Phone Number,Website,Instagram,Food Menu,Table Seating,Wifi,Outlets,Parking,Hours
0,Neat Coffee,"1922 Pomona Ave, Costa Mesa, CA 92627",(949) 873-6328,https://www.neat.coffee/,https://www.instagram.com/neat.coffee/,"Hot food, Baked goods",Yes,Yes,Yes,Yes,"[Monday: 7:00 AM – 4:00 PM, Tuesday: 7:00 AM –..."
1,Moongoat Coffee,"1985 Placentia Ave, Costa Mesa, CA 92627",(949) 612-2875,moongoat.online,https://www.instagram.com/moongoatcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Limited,"[Monday: 6:00 AM – 9:00 PM, Tuesday: 6:00 AM –..."
2,Nüdae Coffee,"1125 Victoria St # U, Costa Mesa, CA 92627",(408) 667-9051,https://www.nudaecoffee.com/,https://www.instagram.com/nudaecoffee/,"Hot food, Baked goods",Limited,Yes,Yes,Yes,"[Monday: 6:30 AM – 5:00 PM, Tuesday: 6:30 AM –..."
3,Lion & Lamb Coffee Roasters,"3186 Pullman St, Costa Mesa, CA 92626",,https://lionandlambcoffee.com/,www.instagram.com/lionandlambcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Yes,"[Monday: 7:00 AM – 5:00 PM, Tuesday: 7:00 AM –..."
4,Work in Progress,"2944 Randolph Av # 1, Costa Mesa, CA 92626",,https://www.wipgoods.com/,https://www.instagram.com/wipgoods/,"Hot food, Baked goods",Limited,Yes,Very limited,Limited,"[Monday: 7:00 AM – 6:00 PM, Tuesday: 7:00 AM –..."
5,In-sīt Coffee,"891 Baker St B19, Costa Mesa, CA 92626",(714) 852-3040,https://www.insitcoffee.com/,https://www.instagram.com/insit_coffee,Baked goods only,Very limited,Yes,Very limited,Limited,"[Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM –..."
6,Buenas Coffee,"150 Paularino Ave Building B, Costa Mesa, CA 9...",,https://www.buenas.coffee/,https://www.instagram.com/buenas.coffee,Baked goods only,Limited,Yes,Yes,Yes,"[Monday: 7:00 AM – 4:00 PM, Tuesday: 7:00 AM –..."
7,Portola Coffee Roasters,"3313 Hyland Ave STE B2, Costa Mesa, CA 92626",(714) 656-7996,https://www.portolacoffee.com/,https://www.instagram.com/portolacoffee,Baked goods only,Yes,Yes,Yes,Yes,"[Monday: 7:00 AM – 5:00 PM, Tuesday: 7:00 AM –..."
8,Philz Coffee,"3420 Bristol St, Costa Mesa, CA 92626",(714) 486-2731,https://philzcoffee.com/,https://www.instagram.com/philzcoffee,"Hot food, Baked goods",Yes,Yes,Yes,Yes,"[Monday: 5:30 AM – 8:00 PM, Tuesday: 5:30 AM –..."


In [10]:
# Connect to PostgreSQL
engine = create_engine(CONN_STR)

In [11]:
df.to_sql('store_page_info', engine, if_exists= 'replace', index= False)

engine.dispose()